# Project 35: Anomaly Detection in Cloud Load Balancer Logs

**Objective:** Build an unsupervised model that can detect anomalous traffic patterns in cloud load balancer logs by establishing a baseline of normal, aggregated behavior (requests per minute, error rates) and identifying significant deviations.

**Dataset Source:** Synthetically Generated (realistic time-series load balancer logs with simulated anomaly events)

**Model:** Isolation Forest for time-series anomaly detection in load balancer metrics

**Instructions:**
This notebook is fully self-contained and does not require external files. Simply run all cells in sequence.

In [ ]:
# ==================================================================================
#  Project 35: Load Balancer Anomaly Detection - Setup and Imports
# ==================================================================================

import pandas as pd
import numpy as np
import random
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime, timedelta

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)

print("All libraries imported successfully.")

In [ ]:
# ==================================================================================
#  Synthetic Load Balancer Log Generation
# ==================================================================================

print("--- Generating Synthetic Aggregated Load Balancer Log Dataset ---")

# Simulation parameters
time_steps_minutes = 1440  # 24 hours of data, aggregated per minute
data = []
anomaly_events = [
    {'start': 1200, 'duration': 60, 'type': '5xx_spike'},    # 20:00 - 21:00: Server errors
    {'start': 300, 'duration': 30, 'type': 'traffic_spike'}, # 05:00 - 05:30: Traffic surge
    {'start': 900, 'duration': 15, 'type': '4xx_spike'}      # 15:00 - 15:15: Client errors
]

print(f"Simulation parameters:")
print(f"• Time period: {time_steps_minutes} minutes (24 hours)")
print(f"• Number of anomaly events: {len(anomaly_events)}")
for i, event in enumerate(anomaly_events):
    start_hour = event['start'] // 60
    start_min = event['start'] % 60
    print(f"  - Event {i+1}: {event['type']} at {start_hour:02d}:{start_min:02d} for {event['duration']} minutes")

for t in range(time_steps_minutes):
    is_anomaly = False
    anomaly_type = 'normal'
    
    # Simulate normal traffic with a daily sinusoidal pattern (peak during the day)
    # Pattern: Low at night (4am = 240 min), peak during day (4pm = 960 min)
    hour_of_day = (t % 1440) / 60  # Convert to hour of day
    daily_pattern = 0.5 + 0.4 * np.sin(2 * np.pi * (hour_of_day - 6) / 24)  # Peak at 2pm
    
    base_requests = int(8000 + 6000 * daily_pattern + np.random.normal(0, 500))
    base_requests = max(1000, base_requests)  # Minimum traffic
    
    # Normal error rates
    http_2xx_rate = 0.98   # Success
    http_4xx_rate = 0.015  # Client-side errors
    http_5xx_rate = 0.005  # Server-side errors
    
    # Check for anomaly events
    for event in anomaly_events:
        if event['start'] <= t < event['start'] + event['duration']:
            is_anomaly = True
            anomaly_type = event['type']
            
            if event['type'] == '5xx_spike':
                # Backend service failure - spike in 5xx errors
                http_5xx_rate = np.random.uniform(0.3, 0.6)  # 30-60% of requests fail
                http_2xx_rate = max(0.1, 1 - http_4xx_rate - http_5xx_rate)
                
            elif event['type'] == 'traffic_spike':
                # Sudden traffic surge (e.g., flash sale, viral content)
                base_requests *= np.random.uniform(5, 10)  # 5-10x traffic
                # Slightly higher error rates due to overload
                http_5xx_rate = 0.02
                http_4xx_rate = 0.03
                http_2xx_rate = 1 - http_4xx_rate - http_5xx_rate
                
            elif event['type'] == '4xx_spike':
                # Client-side issues (e.g., broken API clients, bad deployments)
                http_4xx_rate = np.random.uniform(0.2, 0.4)  # 20-40% client errors
                http_2xx_rate = max(0.3, 1 - http_4xx_rate - http_5xx_rate)
            
            break
    
    # Calculate actual counts
    total_requests = int(base_requests)
    num_2xx = int(total_requests * http_2xx_rate)
    num_4xx = int(total_requests * http_4xx_rate)
    num_5xx = total_requests - num_2xx - num_4xx  # Ensure counts sum to total
    
    # Add some realistic noise to response times
    avg_response_time = np.random.normal(120, 20)  # Average 120ms
    if is_anomaly and anomaly_type in ['5xx_spike', 'traffic_spike']:
        avg_response_time *= np.random.uniform(2, 5)  # Slower during issues
    
    avg_response_time = max(10, avg_response_time)  # Minimum 10ms
    
    data.append([t, total_requests, num_2xx, num_4xx, num_5xx, 
                avg_response_time, is_anomaly, anomaly_type])

df = pd.DataFrame(data, columns=['minute', 'total_requests', '2xx_count', '4xx_count', 
                                '5xx_count', 'avg_response_time_ms', 'is_truly_anomaly', 'anomaly_type'])

# Add timestamp for better visualization
start_time = datetime(2024, 1, 1, 0, 0, 0)
df['timestamp'] = [start_time + timedelta(minutes=i) for i in df['minute']]
df.set_index('minute', inplace=True)

print(f"\nDataset generation complete. Created {len(df)} records.")
print(f"Total anomalous minutes: {df['is_truly_anomaly'].sum()}")
print(f"Percentage of anomalous data: {(df['is_truly_anomaly'].sum() / len(df)) * 100:.2f}%")

print("\nAnomaly event summary:")
anomaly_summary = df[df['is_truly_anomaly'] == True]['anomaly_type'].value_counts()
print(anomaly_summary)

print("\nDataset Sample:")
print(df.sample(10).round(2))

In [ ]:
# ==================================================================================
#  Feature Engineering
# ==================================================================================

print("--- Engineering Rate-Based Features ---")

# Raw counts can be misleading. Rates are often better features for anomaly detection.
df['5xx_error_rate'] = df['5xx_count'] / df['total_requests']
df['4xx_error_rate'] = df['4xx_count'] / df['total_requests']
df['2xx_success_rate'] = df['2xx_count'] / df['total_requests']
df['total_error_rate'] = (df['4xx_count'] + df['5xx_count']) / df['total_requests']

# Handle potential division by zero if total_requests is 0
df.fillna(0, inplace=True)

# Add derived features
df['requests_per_hour'] = df['total_requests'] * 60  # Scale up to hourly
df['errors_per_minute'] = df['4xx_count'] + df['5xx_count']

# Create rolling statistics to capture trends
window_size = 10  # 10-minute rolling window
df['total_requests_rolling_mean'] = df['total_requests'].rolling(window=window_size, min_periods=1).mean()
df['5xx_rate_rolling_mean'] = df['5xx_error_rate'].rolling(window=window_size, min_periods=1).mean()
df['response_time_rolling_mean'] = df['avg_response_time_ms'].rolling(window=window_size, min_periods=1).mean()

# Select features for anomaly detection
feature_cols = [
    'total_requests', '5xx_error_rate', '4xx_error_rate', 'total_error_rate',
    'avg_response_time_ms', 'errors_per_minute'
]

X = df[feature_cols].copy()

print(f"\nFeatures engineered:")
for col in df.columns:
    if col.endswith('_rate') or col.endswith('_mean') or col in feature_cols:
        print(f"• {col}")

print(f"\nFeature matrix shape: {X.shape}")
print(f"Features for anomaly detection: {feature_cols}")

print(f"\nFeature statistics:")
print(X.describe().round(4))

# Check for any problematic values
print(f"\nData quality check:")
print(f"• Missing values: {X.isnull().sum().sum()}")
print(f"• Infinite values: {np.isinf(X.values).sum()}")
print(f"• Negative values in error rates: {(X[['5xx_error_rate', '4xx_error_rate', 'total_error_rate']] < 0).sum().sum()}")

In [ ]:
# ==================================================================================
#  Data Exploration and Visualization
# ==================================================================================

print("--- Load Balancer Data Exploration ---")

# Create comprehensive time series visualization
fig, axes = plt.subplots(3, 2, figsize=(18, 16))
fig.suptitle('Load Balancer Traffic Analysis - 24 Hour Period', fontsize=16)

# 1. Total requests over time
axes[0,0].plot(df.index, df['total_requests'], alpha=0.7, color='blue')
# Highlight anomaly periods
anomaly_data = df[df['is_truly_anomaly'] == True]
axes[0,0].scatter(anomaly_data.index, anomaly_data['total_requests'], 
                 color='red', s=30, alpha=0.8, label='Anomalies')
axes[0,0].set_title('Total Requests per Minute')
axes[0,0].set_ylabel('Requests/min')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# 2. Error rates over time
axes[0,1].plot(df.index, df['5xx_error_rate'], alpha=0.8, color='red', label='5xx Error Rate')
axes[0,1].plot(df.index, df['4xx_error_rate'], alpha=0.8, color='orange', label='4xx Error Rate')
axes[0,1].scatter(anomaly_data.index, anomaly_data['5xx_error_rate'], 
                 color='darkred', s=30, alpha=0.8)
axes[0,1].set_title('Error Rates Over Time')
axes[0,1].set_ylabel('Error Rate')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# 3. Response time over time
axes[1,0].plot(df.index, df['avg_response_time_ms'], alpha=0.7, color='green')
axes[1,0].scatter(anomaly_data.index, anomaly_data['avg_response_time_ms'], 
                 color='red', s=30, alpha=0.8, label='Anomalies')
axes[1,0].set_title('Average Response Time')
axes[1,0].set_ylabel('Response Time (ms)')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# 4. Distribution of key metrics
normal_data = df[df['is_truly_anomaly'] == False]
axes[1,1].hist(normal_data['5xx_error_rate'], bins=50, alpha=0.7, 
              label='Normal', color='blue', density=True)
axes[1,1].hist(anomaly_data['5xx_error_rate'], bins=20, alpha=0.7, 
              label='Anomalous', color='red', density=True)
axes[1,1].set_title('Distribution of 5xx Error Rates')
axes[1,1].set_xlabel('5xx Error Rate')
axes[1,1].set_ylabel('Density')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

# 5. HTTP status code breakdown over time (stacked area)
axes[2,0].fill_between(df.index, 0, df['2xx_count'], alpha=0.7, color='green', label='2xx Success')
axes[2,0].fill_between(df.index, df['2xx_count'], df['2xx_count'] + df['4xx_count'], 
                      alpha=0.7, color='orange', label='4xx Client Error')
axes[2,0].fill_between(df.index, df['2xx_count'] + df['4xx_count'], 
                      df['2xx_count'] + df['4xx_count'] + df['5xx_count'], 
                      alpha=0.7, color='red', label='5xx Server Error')
axes[2,0].set_title('HTTP Status Code Breakdown Over Time')
axes[2,0].set_xlabel('Time (minutes)')
axes[2,0].set_ylabel('Request Count')
axes[2,0].legend()
axes[2,0].grid(True, alpha=0.3)

# 6. Feature correlation heatmap
correlation_matrix = X.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
           fmt='.2f', ax=axes[2,1])
axes[2,1].set_title('Feature Correlation Matrix')

plt.tight_layout()
plt.show()

# Print anomaly event details
print("\nAnomaly Event Details:")
for event_type in df[df['is_truly_anomaly'] == True]['anomaly_type'].unique():
    event_data = df[df['anomaly_type'] == event_type]
    print(f"\n{event_type.upper()}:")
    print(f"  Duration: {len(event_data)} minutes")
    print(f"  Avg requests/min: {event_data['total_requests'].mean():.0f}")
    print(f"  Avg 5xx error rate: {event_data['5xx_error_rate'].mean():.3f}")
    print(f"  Avg 4xx error rate: {event_data['4xx_error_rate'].mean():.3f}")
    print(f"  Avg response time: {event_data['avg_response_time_ms'].mean():.1f} ms")

# Traffic pattern analysis
print(f"\nTraffic Pattern Analysis:")
print(f"• Peak traffic: {df['total_requests'].max():,} requests/min")
print(f"• Average traffic: {df['total_requests'].mean():.0f} requests/min")
print(f"• Minimum traffic: {df['total_requests'].min():,} requests/min")
print(f"• Normal 5xx error rate: {normal_data['5xx_error_rate'].mean():.4f} ({normal_data['5xx_error_rate'].mean()*100:.2f}%)")
print(f"• Normal 4xx error rate: {normal_data['4xx_error_rate'].mean():.4f} ({normal_data['4xx_error_rate'].mean()*100:.2f}%)")
print(f"• Normal response time: {normal_data['avg_response_time_ms'].mean():.1f} ms")

In [ ]:
# ==================================================================================
#  Unsupervised Model Training
# ==================================================================================

print("--- Unsupervised Model Training (on NORMAL data only) ---")

# For realistic anomaly detection, we should train only on normal data
# In practice, we'd use a known-good time period
normal_period_end = 1000  # Train on first 1000 minutes (16.67 hours)
training_data = df.iloc[:normal_period_end]
normal_training_data = training_data[training_data['is_truly_anomaly'] == False]

print(f"Training approach:")
print(f"• Using first {normal_period_end} minutes for training")
print(f"• Training samples: {len(normal_training_data)} (normal traffic only)")
print(f"• Test samples: {len(df) - normal_period_end}")
print(f"• Anomalies in training period: {training_data['is_truly_anomaly'].sum()}")

# Prepare training features
X_train = normal_training_data[feature_cols].copy()

# Handle any edge cases
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_train = X_train.fillna(X_train.median())

print(f"\nTraining data statistics:")
print(X_train.describe().round(4))

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

print(f"\nFeatures scaled using StandardScaler")

# Train Isolation Forest on normal data
# Use a conservative contamination rate since we're training on normal data
contamination_rate = 0.05  # Allow for 5% outliers in "normal" data

model = IsolationForest(
    n_estimators=200,
    contamination=contamination_rate,
    random_state=42,
    n_jobs=-1
)

print(f"\nIsolation Forest Configuration:")
print(f"• Number of estimators: {model.n_estimators}")
print(f"• Contamination rate: {model.contamination} ({model.contamination*100:.1f}%)")
print(f"• Training on normal data only: Yes")

print("\nTraining the Isolation Forest model...")
start_time = time.time()
model.fit(X_train_scaled)
end_time = time.time()
print(f"Training completed in {end_time - start_time:.2f} seconds.")

# Now apply the model to the entire dataset
X_all = df[feature_cols].copy()
X_all = X_all.replace([np.inf, -np.inf], np.nan)
X_all = X_all.fillna(X_all.median())
X_all_scaled = scaler.transform(X_all)

# Make predictions on entire dataset
y_pred_raw = model.predict(X_all_scaled)
y_pred = (y_pred_raw == -1).astype(int)  # Convert to binary (0=normal, 1=anomaly)
anomaly_scores = model.decision_function(X_all_scaled)

print(f"\nPrediction Summary:")
print(f"• Total samples: {len(y_pred)}")
print(f"• Predicted anomalies: {np.sum(y_pred)} ({np.mean(y_pred)*100:.2f}%)")
print(f"• Actual anomalies: {df['is_truly_anomaly'].sum()} ({df['is_truly_anomaly'].mean()*100:.2f}%)")
print(f"• Anomaly score range: [{np.min(anomaly_scores):.3f}, {np.max(anomaly_scores):.3f}]")

In [ ]:
# ==================================================================================
#  Model Evaluation
# ==================================================================================

print("--- Model Evaluation ---")

# Evaluate against ground truth
y_true = df['is_truly_anomaly'].values

# Calculate evaluation metrics
precision, recall, f1, support = precision_recall_fscore_support(
    y_true, y_pred, average='binary', pos_label=1
)

print(f"\nBinary Classification Metrics:")
print(f"• Precision: {precision:.3f} (of predicted anomalies, what % were actually anomalies)")
print(f"• Recall: {recall:.3f} (of actual anomalies, what % were detected)")
print(f"• F1-Score: {f1:.3f} (harmonic mean of precision and recall)")

# Detailed classification report
print("\nDetailed Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Normal', 'Anomalous']))

# Confusion matrix analysis
cm = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = cm.ravel()

accuracy = (tp + tn) / (tp + tn + fp + fn)
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
false_positive_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
false_negative_rate = fn / (fn + tp) if (fn + tp) > 0 else 0

print(f"\nConfusion Matrix Analysis:")
print(f"                 Predicted")
print(f"Actual    Normal  Anomaly")
print(f"Normal    {tn:6d}  {fp:7d}")
print(f"Anomaly   {fn:6d}  {tp:7d}")

print(f"\nAdditional Metrics:")
print(f"• Accuracy: {accuracy:.3f}")
print(f"• Specificity: {specificity:.3f} (true negative rate)")
print(f"• False Positive Rate: {false_positive_rate:.3f}")
print(f"• False Negative Rate: {false_negative_rate:.3f}")

# Evaluate by anomaly type
print(f"\nDetection Performance by Anomaly Type:")
for anomaly_type in df[df['is_truly_anomaly'] == True]['anomaly_type'].unique():
    type_mask = df['anomaly_type'] == anomaly_type
    type_true = y_true[type_mask]
    type_pred = y_pred[type_mask]
    
    if len(type_true) > 0:
        type_precision = np.sum((type_true == 1) & (type_pred == 1)) / np.sum(type_pred == 1) if np.sum(type_pred == 1) > 0 else 0
        type_recall = np.sum((type_true == 1) & (type_pred == 1)) / np.sum(type_true == 1) if np.sum(type_true == 1) > 0 else 0
        
        print(f"  {anomaly_type}:")
        print(f"    • Total events: {np.sum(type_true == 1)}")
        print(f"    • Detected: {np.sum((type_true == 1) & (type_pred == 1))}")
        print(f"    • Recall: {type_recall:.3f}")
        print(f"    • Precision: {type_precision:.3f}")

# Timing analysis
print(f"\nTiming Analysis:")
# Calculate detection delay (how quickly anomalies are detected)
detection_delays = []
for event in anomaly_events:
    event_range = range(event['start'], event['start'] + event['duration'])
    event_predictions = y_pred[event['start']:event['start'] + event['duration']]
    
    if np.any(event_predictions == 1):
        first_detection = np.argmax(event_predictions == 1)
        detection_delays.append(first_detection)
        print(f"  {event['type']}: Detected after {first_detection} minutes")
    else:
        print(f"  {event['type']}: Not detected")

if detection_delays:
    print(f"  Average detection delay: {np.mean(detection_delays):.1f} minutes")

In [ ]:
# ==================================================================================
#  Results Visualization
# ==================================================================================

print("--- Results Visualization ---")

# Add predictions to dataframe for visualization
df_viz = df.copy()
df_viz['predicted_anomaly'] = y_pred
df_viz['anomaly_score'] = anomaly_scores

# Create comprehensive results visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 12))
fig.suptitle('Load Balancer Anomaly Detection Results', fontsize=16)

# 1. Time series with predictions
ax1.plot(df_viz.index, df_viz['total_requests'], alpha=0.7, color='blue', label='Traffic')

# Highlight true anomalies
true_anomalies = df_viz[df_viz['is_truly_anomaly'] == True]
ax1.scatter(true_anomalies.index, true_anomalies['total_requests'], 
           color='red', s=50, label='True Anomalies', zorder=5)

# Highlight detected anomalies
detected_anomalies = df_viz[df_viz['predicted_anomaly'] == 1]
ax1.scatter(detected_anomalies.index, detected_anomalies['total_requests'], 
           color='orange', s=30, marker='x', label='Detected Anomalies', zorder=5)

# Mark training period
ax1.axvline(normal_period_end, color='green', linestyle='--', alpha=0.7, label='Training Cutoff')

ax1.set_title('Anomaly Detection Results - Traffic Volume')
ax1.set_xlabel('Time (minutes)')
ax1.set_ylabel('Requests per Minute')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Error rate time series
ax2.plot(df_viz.index, df_viz['5xx_error_rate'], alpha=0.7, color='red', label='5xx Error Rate')
ax2.scatter(true_anomalies.index, true_anomalies['5xx_error_rate'], 
           color='darkred', s=50, label='True Anomalies', zorder=5)
ax2.scatter(detected_anomalies.index, detected_anomalies['5xx_error_rate'], 
           color='orange', s=30, marker='x', label='Detected Anomalies', zorder=5)
ax2.axvline(normal_period_end, color='green', linestyle='--', alpha=0.7, label='Training Cutoff')

ax2.set_title('Anomaly Detection Results - 5xx Error Rate')
ax2.set_xlabel('Time (minutes)')
ax2.set_ylabel('5xx Error Rate')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Anomaly scores distribution
normal_scores = anomaly_scores[y_true == 0]
anomaly_scores_true = anomaly_scores[y_true == 1]

ax3.hist(normal_scores, bins=50, alpha=0.7, label='Normal', color='blue', density=True)
ax3.hist(anomaly_scores_true, bins=30, alpha=0.7, label='True Anomalies', color='red', density=True)
ax3.axvline(model.offset_, color='green', linestyle='--', linewidth=2, label='Decision Threshold')
ax3.set_title('Anomaly Scores Distribution')
ax3.set_xlabel('Anomaly Score')
ax3.set_ylabel('Density')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Confusion matrix heatmap
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Normal', 'Anomalous'], 
            yticklabels=['Normal', 'Anomalous'], ax=ax4)
ax4.set_title('Confusion Matrix')
ax4.set_ylabel('Actual')
ax4.set_xlabel('Predicted')

plt.tight_layout()
plt.show()

# Feature importance analysis (approximation)
print("\n--- Feature Importance Analysis ---")
feature_importance = []

for i, feature in enumerate(feature_cols):
    # Calculate correlation between feature and anomaly detection
    feature_values = X_all.iloc[:, i].values
    correlation = np.corrcoef(feature_values, y_pred)[0, 1]
    feature_importance.append(abs(correlation))

importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("\nFeature Importance (correlation with anomaly predictions):")
print(importance_df.round(4))

# Plot feature importance
plt.figure(figsize=(10, 6))
sns.barplot(data=importance_df, x='importance', y='feature')
plt.title('Feature Importance for Anomaly Detection')
plt.xlabel('Absolute Correlation with Predictions')
plt.tight_layout()
plt.show()

In [ ]:
# ==================================================================================
#  Alert Analysis and Operational Insights
# ==================================================================================

print("--- Alert Analysis and Operational Insights ---")

# Analyze alert patterns
alerts_df = df_viz[df_viz['predicted_anomaly'] == 1].copy()
print(f"\nAlert Summary:")
print(f"• Total alerts generated: {len(alerts_df)}")
print(f"• True positive alerts: {np.sum((df_viz['predicted_anomaly'] == 1) & (df_viz['is_truly_anomaly'] == True))}")
print(f"• False positive alerts: {np.sum((df_viz['predicted_anomaly'] == 1) & (df_viz['is_truly_anomaly'] == False))}")

if len(alerts_df) > 0:
    print(f"\nAlert Characteristics:")
    print(f"• Average requests/min during alerts: {alerts_df['total_requests'].mean():.0f}")
    print(f"• Average 5xx error rate during alerts: {alerts_df['5xx_error_rate'].mean():.3f}")
    print(f"• Average 4xx error rate during alerts: {alerts_df['4xx_error_rate'].mean():.3f}")
    print(f"• Average response time during alerts: {alerts_df['avg_response_time_ms'].mean():.1f} ms")
    print(f"• Average anomaly score: {alerts_df['anomaly_score'].mean():.3f}")

# Alert clustering (consecutive alerts)
alert_periods = []
if len(alerts_df) > 0:
    alert_indices = alerts_df.index.tolist()
    current_period = [alert_indices[0]]
    
    for i in range(1, len(alert_indices)):
        if alert_indices[i] - alert_indices[i-1] <= 2:  # Within 2 minutes
            current_period.append(alert_indices[i])
        else:
            alert_periods.append(current_period)
            current_period = [alert_indices[i]]
    
    alert_periods.append(current_period)

print(f"\nAlert Period Analysis:")
print(f"• Number of distinct alert periods: {len(alert_periods)}")

for i, period in enumerate(alert_periods):
    start_time = period[0]
    end_time = period[-1]
    duration = end_time - start_time + 1
    
    period_data = df_viz.loc[period]
    true_positives = period_data['is_truly_anomaly'].sum()
    
    print(f"  Period {i+1}: Minutes {start_time}-{end_time} (duration: {duration} min)")
    print(f"    • True anomalies in period: {true_positives}")
    print(f"    • Peak requests: {period_data['total_requests'].max():,}")
    print(f"    • Peak 5xx rate: {period_data['5xx_error_rate'].max():.3f}")
    print(f"    • Min anomaly score: {period_data['anomaly_score'].min():.3f}")

# Missed anomaly analysis
missed_anomalies = df_viz[(df_viz['is_truly_anomaly'] == True) & (df_viz['predicted_anomaly'] == 0)]
print(f"\nMissed Anomaly Analysis:")
print(f"• Total missed anomalies: {len(missed_anomalies)}")

if len(missed_anomalies) > 0:
    print(f"• Missed anomaly types:")
    missed_types = missed_anomalies['anomaly_type'].value_counts()
    for anomaly_type, count in missed_types.items():
        print(f"  - {anomaly_type}: {count} instances")
    
    print(f"\nCharacteristics of missed anomalies:")
    print(f"• Average requests/min: {missed_anomalies['total_requests'].mean():.0f}")
    print(f"• Average 5xx error rate: {missed_anomalies['5xx_error_rate'].mean():.3f}")
    print(f"• Average anomaly score: {missed_anomalies['anomaly_score'].mean():.3f}")
    print(f"• Anomaly score range: [{missed_anomalies['anomaly_score'].min():.3f}, {missed_anomalies['anomaly_score'].max():.3f}]")

# Operational recommendations
print(f"\nOperational Recommendations:")
if false_positive_rate < 0.05:
    print(f"• Low false positive rate ({false_positive_rate:.1%}) - model is ready for production alerting")
else:
    print(f"• Consider tuning threshold to reduce false positive rate ({false_positive_rate:.1%})")

if recall > 0.8:
    print(f"• Excellent anomaly detection rate ({recall:.1%}) - catches most incidents")
else:
    print(f"• Consider adjusting contamination parameter to improve detection rate ({recall:.1%})")

print(f"• Average detection delay: {np.mean(detection_delays):.1f} minutes" if detection_delays else "• Some anomaly types not detected")
print(f"• Recommended alert aggregation window: 2-3 minutes to reduce noise")
print(f"• Consider escalation for sustained alerts (>10 minutes)")

In [ ]:
# ==================================================================================
#  Conclusion
# ==================================================================================

print("--- Conclusion ---")
print("The Isolation Forest model successfully learned to detect anomalous patterns in load balancer traffic.")

print("\nKey Performance Results:")
print(f"• Overall accuracy: {accuracy:.3f} ({accuracy*100:.1f}%)")
print(f"• Precision: {precision:.3f} (reliability of anomaly alerts)")
print(f"• Recall: {recall:.3f} (coverage of actual anomalies)")
print(f"• F1-Score: {f1:.3f} (balanced performance metric)")
print(f"• False positive rate: {false_positive_rate:.3f} ({false_positive_rate*100:.1f}%)")
print(f"• Training time: {end_time - start_time:.2f} seconds")

print("\nDetection Effectiveness by Anomaly Type:")
for anomaly_type in df[df['is_truly_anomaly'] == True]['anomaly_type'].unique():
    type_data = df[df['anomaly_type'] == anomaly_type]
    type_detected = np.sum((df['anomaly_type'] == anomaly_type) & (df_viz['predicted_anomaly'] == 1))
    detection_rate = type_detected / len(type_data) if len(type_data) > 0 else 0
    print(f"• {anomaly_type}: {detection_rate:.1%} detection rate ({type_detected}/{len(type_data)} events)")

print("\nBusiness Impact:")
print("• **Proactive Incident Detection**: Identify service degradations before customer impact")
print("• **Reduced MTTR**: Faster detection leads to quicker incident response")
print("• **SLA Protection**: Prevent cascading failures through early warning")
print("• **Operational Efficiency**: Automated monitoring reduces manual oversight")

print("\nOperational Applications:")
print("• **Real-time Alerting**: Deploy for continuous load balancer monitoring")
print("• **Auto-scaling Triggers**: Use traffic spike detection for infrastructure scaling")
print("• **Health Dashboards**: Integrate anomaly scores into monitoring dashboards")
print("• **Runbook Automation**: Trigger automated responses for specific anomaly types")

print("\nTechnical Insights:")
print(f"• Most important features: {', '.join(importance_df.head(3)['feature'].tolist())}")
print("• Error rates more indicative than absolute request counts")
print("• Response time patterns provide additional anomaly signals")
print("• Training on normal data improves real-world applicability")

print("\nModel Strengths:")
print("• Unsupervised approach requires no labeled anomaly data")
print("• Adapts to normal traffic patterns and seasonal variations")
print("• Low computational overhead suitable for real-time deployment")
print("• Provides interpretable anomaly scores for alert prioritization")

print("\nProduction Deployment Strategy:")
print("• **Gradual Rollout**: Start with shadow mode to validate alerts")
print("• **Threshold Tuning**: Adjust based on operational feedback")
print("• **Alert Aggregation**: Group consecutive alerts to reduce noise")
print("• **Feedback Loop**: Incorporate operator feedback to improve accuracy")

print("\nExtensions and Improvements:")
print("• **Multi-metric Fusion**: Combine with infrastructure metrics (CPU, memory)")
print("• **Temporal Models**: Add time-series specific algorithms (LSTM, Prophet)")
print("• **Ensemble Methods**: Combine multiple anomaly detection approaches")
print("• **Adaptive Thresholds**: Dynamic adjustment based on traffic patterns")

print(f"\nRecommendations:")
if precision > 0.8 and recall > 0.7:
    print("• Model performance excellent - ready for production deployment")
elif precision > 0.7:
    print("• Good precision but consider improving recall for better anomaly coverage")
elif recall > 0.7:
    print("• Good recall but consider reducing false positives for operational efficiency")
else:
    print("• Consider additional feature engineering or model tuning")

print("• Implement alert fatigue prevention through intelligent grouping")
print("• Establish escalation procedures for sustained anomalies")
print("• Regular model retraining to adapt to evolving traffic patterns")
print("• Integration with incident management systems for automated ticket creation")

print(f"\nFinal Assessment:")
print(f"• The model demonstrates {precision:.1%} precision and {recall:.1%} recall")
print(f"• Average detection delay of {np.mean(detection_delays):.1f} minutes enables rapid response" if detection_delays else "• Some improvement needed in detection speed")
print(f"• Low false positive rate ({false_positive_rate:.1%}) minimizes alert fatigue")
print(f"• Ready for integration with production monitoring infrastructure")